In [1]:
%load_ext autoreload
%env PYTHONPATH=../

import sys
sys.path.append("../")
import pprint
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import numpy as np
from tensorflow import keras
from oodeel.methods import MLS, DKNN, ODIN
from oodeel.methods import DKNN
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from oodeel.eval.metrics import bench_metrics, get_curve
from oodeel.datasets import OODDataset

from sklearn.metrics import *

import warnings
warnings.filterwarnings("ignore")

pp = pprint.PrettyPrinter()

env: PYTHONPATH=../


/Users/paul/workspace/oodeel/oodeel_dev_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Two datasets experiment

In [2]:
%autoreload 2

oods_in = OODDataset('mnist', split="test")
oods_out = OODDataset('fashion_mnist', split="test")
oods_fit = OODDataset('mnist', split="train")

def preprocess_fn(*inputs):
    x = inputs[0] / 255
    return tuple([x] + list(inputs[1:]))


batch_size = 128
ds_in = oods_in.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn)
ds_out = oods_out.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn)
ds_fit = oods_fit.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn, shuffle=True)

Metal device set to: AMD Radeon Pro 5500M

systemMemory: 64.00 GB
maxCacheSize: 3.99 GB

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [3]:

model = tf.keras.models.load_model("../saved_models/mnist_model")

### MLS

In [4]:

%autoreload 2


oodmodel = MLS()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)


{'accuracy_score': 0.9629,
 'auroc': 0.994483825,
 'fpr95tpr': 0.023,
 'roc_auc_score': 0.99448376}


### DKNN

In [5]:
%autoreload 2


oodmodel = DKNN()
oodmodel.fit(model, ds_fit.take(10000))
scores_in = oodmodel.score(ds_in.take(1000))
scores_out = oodmodel.score(ds_out.take(1000))


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.5,
 'auroc': 0.9986808250000001,
 'fpr95tpr': 0.0044,
 'roc_auc_score': 0.99868077}


### ODIN

In [6]:

%autoreload 2
from oodeel.methods import ODIN

oodmodel = ODIN()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.5,
 'auroc': 0.9942106399999999,
 'fpr95tpr': 0.0263,
 'roc_auc_score': 0.9942105549999999}


## Single dataset experiment

(Leave-$k$-classes-out training).
First, we need to define a training function

In [7]:

%autoreload 2

oods_test = OODDataset('mnist', split="test")
oods_train = OODDataset('mnist', split="train")

batch_size = 128
inc_labels = [0, 1, 2, 3, 4]
oods_train, _ = oods_train.assign_ood_labels_by_class(in_labels=inc_labels)
oods_in, oods_out = oods_test.assign_ood_labels_by_class(in_labels=inc_labels)


def preprocess_fn(*inputs):
    x = inputs[0] / 255
    return tuple([x] + list(inputs[1:]))

ds_train = oods_train.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn, shuffle=True)
ds_in = oods_in.prepare(batch_size=batch_size, with_ood_labels=False, preprocess_fn=preprocess_fn)
ds_out = oods_out.prepare(batch_size=batch_size, with_ood_labels=False, preprocess_fn=preprocess_fn)


In [8]:
%autoreload 2
from oodeel.models.training_funs import train_convnet_classifier

train_config = {
    "input_shape": (28, 28, 1),
    "num_classes": len(inc_labels),
    "is_prepared": True,
    "batch_size": 128,
    "epochs": 5
}

model = train_convnet_classifier(ds_train, **train_config)

Epoch 1/5
240/240 [==============================] - 7s 23ms/step - loss: 0.2208 - accuracy: 0.9340
Epoch 2/5
240/240 [==============================] - 5s 19ms/step - loss: 0.0484 - accuracy: 0.9857
Epoch 3/5
240/240 [==============================] - 4s 18ms/step - loss: 0.0392 - accuracy: 0.9879
Epoch 4/5
240/240 [==============================] - 4s 18ms/step - loss: 0.0359 - accuracy: 0.9893
Epoch 5/5
240/240 [==============================] - 4s 18ms/step - loss: 0.0367 - accuracy: 0.9888


## MLS

In [9]:
%autoreload 2

oodmodel = MLS()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.8767,
 'auroc': 0.9425724977291451,
 'fpr95tpr': 0.2759291690990465,
 'roc_auc_score': 0.9425728580075826}


### DKNN

In [10]:
%autoreload 2

oodmodel = DKNN()
oodmodel.fit(model, ds_train.take(10000))
scores_in = oodmodel.score(ds_in.take(1000))
scores_out = oodmodel.score(ds_out.take(1000))


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.4861,
 'auroc': 0.9734971175122983,
 'fpr95tpr': 0.11325160537069469,
 'roc_auc_score': 0.9734962168162041}


### ODIN

In [11]:
%autoreload 2

#x_test, y_id = data_handler.convert_to_numpy(x_id)

oodmodel = ODIN()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.4861,
 'auroc': 0.9118010963593104,
 'fpr95tpr': 0.40358046312512164,
 'roc_auc_score': 0.9118018969780606}
